In [147]:
!pip install -q lxml

import bs4
import lxml
import pandas
import urllib

from urllib import request

## Exercice 1 : Scrap a table on a wikipedia page

We would like to display on a map the location of Summer Olympic Games since 1896. We will use a [Wikipedia page](https://fr.wikipedia.org/wiki/Jeux_olympiques) to scrap the associated table.
Below is the code to extract the content of the page using `request` and display its title using `BeautifulSoup`.

In [148]:
jo = "https://fr.wikipedia.org/wiki/Jeux_olympiques"

request_text = request.urlopen(jo).read()
print(request_text[:1000])
page = bs4.BeautifulSoup(request_text, "lxml")
print(page.find("title"))

b'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-0 vector-feature-client-preferences-disabled vector-feature-client-prefs-pinned-disabled vector-toc-available" lang="fr" dir="ltr">\n<head>\n<meta charset="UTF-8">\n<title>Jeux olympiques \xe2\x80\x94 Wikip\xc3\xa9dia</title>\n<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 ve

# Web Scrapping in Python


**TODO pour moi avant le TME**

1.   Faire la version étudiant en enlevant les réponses. Ne garder qu'un code générique pour la carte qu'ils devront adapter



The first objective of this notebook is to discover the `request` and `BeautifulSoup` libraries to crawl a table on a Wikitable page, build a dataframe, and create a map.

*   `request` and [urllib](https://docs.python.org/3/library/urllib.html#module-urllib) for requestion REST API
*   [BeautifulSoup4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) to inspect webpages

Please note that this part serves as an initiation of web scrapping but you will need to learn by yourself to make the project. It is inspired from notebooks published by Galiana, Lino. 2023. Python Pour La Data Science. https://doi.org/10.5281/zenodo.8229676. Exercices target different sources of information, the code needs to be adapted.











Our objective here is to extract the different information in the first table "Jeux olympiques d'été" and to build a data frame.

To proceed, you will have to follow these steps:
*   Find the table
*   Collect each line of the table
*   Retrieve the different columns and transform them into text format. Also, use `strip` to format the value into a proper text format (e.g., without useless spaces). Store these lines (formated as a table of columns) in a table.

**Warning: We have two location for 1940. You have therefore to manage this case: identify the attribute indicating this fact and write a code to collect those lines and insert in the previous table in the right place**

* Collect headers of the HTML table
* Build a data frame from the result table and the headers








In [82]:
import re
# recupere le tableau des jeux olympiques
table = page.find("table", attrs="wikitable center")

# recupere le titre du tableau
tab_title = table.a.text

def processing(line) : 
    line = re.sub(r"\[.*\]|\(.*\)", "", line)
    line = line.split("\n")
    line = [x for x in line if x != ""]

    return line


def find_nb_ligne(tr):
    rows = 0
    td = tr.find("td")
    if td is not None and td.get('rowspan') is not None:
        rows = td.get('rowspan')
    return rows


# recupere les lignes du tableau et les stocker dans une liste
list_ligne = []
nb_rows = -1
entete = []
for ligne in table.find_all("tr"):
    process_line = processing(ligne.text)

    if nb_rows != 0 and int(find_nb_ligne(ligne))  != 0:
        nb_rows = int(find_nb_ligne(ligne))
        entete = process_line[0:2]
        list_ligne.append(process_line)
        nb_rows -= 1
    else:
        if nb_rows > 0:
            process_line = entete + process_line
            list_ligne.append(process_line)
            nb_rows -= 1
        else:
            list_ligne.append(process_line)
    

#enteete du tableau
tab_header = list_ligne[0]
list_ligne



[['Année', 'Olympiade', 'Ville hôte', 'Pays', 'Continent'],
 ['1896', 'I', 'Athènes ', ' Grèce ', 'Europe '],
 ['1900', 'II', 'Paris ', ' France ', 'Europe '],
 ['1904', 'III', 'Saint-Louis ', ' États-Unis ', 'Amérique du Nord '],
 ['1906', '• ', 'Athènes ', ' Grèce ', 'Europe'],
 ['1908', 'IV', 'Londres ', ' Royaume-Uni ', 'Europe '],
 ['1912', 'V', 'Stockholm ', ' Suède ', 'Europe '],
 ['1916', 'VI ', 'Berlin', ' Allemagne', 'Europe'],
 ['1920', 'VII', 'Anvers ', ' Belgique ', 'Europe '],
 ['1924', 'VIII', 'Paris ', ' France ', 'Europe '],
 ['1928', 'IX', 'Amsterdam ', ' Pays-Bas ', 'Europe '],
 ['1932', 'X', 'Los Angeles ', ' États-Unis ', 'Amérique du Nord '],
 ['1936', 'XI', 'Berlin ', ' Allemagne ', 'Europe '],
 ['1940', 'XII ', 'Tokyo', ' Japon', 'Asie'],
 ['1940', 'XII ', 'Helsinki', ' Finlande', 'Europe'],
 ['1944', 'XIII ', 'Londres', ' Royaume-Uni', 'Europe'],
 ['1948', 'XIV', 'Londres ', ' Royaume-Uni ', 'Europe '],
 ['1952', 'XV', 'Helsinki ', ' Finlande ', 'Europe '],
 ['

In [84]:
print(tab_title)
df = pandas.DataFrame(list_ligne[1:], columns=tab_header)
df

Jeux olympiques d'été


,Année,Olympiade,Ville hôte,Pays,Continent
0,1896,I,Athènes,Grèce,Europe
1,1900,II,Paris,France,Europe
2,1904,III,Saint-Louis,États-Unis,Amérique du Nord
3,1906,•,Athènes,Grèce,Europe
4,1908,IV,Londres,Royaume-Uni,Europe
5,1912,V,Stockholm,Suède,Europe
6,1916,VI,Berlin,Allemagne,Europe
7,1920,VII,Anvers,Belgique,Europe
8,1924,VIII,Paris,France,Europe
9,1928,IX,Amsterdam,Pays-Bas,Europe


## Exercice 2: Locate places on a map

The objective here is to identify organizer cities on a map.
You will have to code the following steps:
* Collect the URL of each city
* Go the this page using `urllib.request`
* Find the coordinates
* Store cities and coordinates in a data frame

In [130]:
import requests
from lxml import html
jo = "https://fr.wikipedia.org/wiki/Jeux_olympiques"

response = requests.get(jo)
page_for_links = html.fromstring(response.content)

links = page_for_links.xpath('//table[@class="wikitable center" and caption[contains(a, "Jeux olympiques d\'été")]]//tr/td[3]//a/@href')
links = ["https://fr.wikipedia.org/" + x for x in links]
print (links)


['https://fr.wikipedia.org//wiki/Ath%C3%A8nes', 'https://fr.wikipedia.org//wiki/Paris', 'https://fr.wikipedia.org//wiki/Saint-Louis_(Missouri)', 'https://fr.wikipedia.org//wiki/Ath%C3%A8nes', 'https://fr.wikipedia.org//wiki/Londres', 'https://fr.wikipedia.org//wiki/Stockholm', 'https://fr.wikipedia.org//wiki/Berlin', 'https://fr.wikipedia.org//wiki/Anvers', 'https://fr.wikipedia.org//wiki/Paris', 'https://fr.wikipedia.org//wiki/Amsterdam', 'https://fr.wikipedia.org//wiki/Los_Angeles', 'https://fr.wikipedia.org//wiki/Berlin', 'https://fr.wikipedia.org//wiki/Tokyo', 'https://fr.wikipedia.org//wiki/Londres', 'https://fr.wikipedia.org//wiki/Londres', 'https://fr.wikipedia.org//wiki/Helsinki', 'https://fr.wikipedia.org//wiki/Melbourne', 'https://fr.wikipedia.org//wiki/Rome', 'https://fr.wikipedia.org//wiki/Tokyo', 'https://fr.wikipedia.org//wiki/Mexico', 'https://fr.wikipedia.org//wiki/Munich', 'https://fr.wikipedia.org//wiki/Montr%C3%A9al', 'https://fr.wikipedia.org//wiki/Moscou', 'https:/

In [142]:
Coordonnees = []
for link in links:
    response = requests.get(link)
    page_for_links = html.fromstring(response.content)
    coord = page_for_links.xpath('//tr[th[contains(a, "Coordonnées")]]/td/a/text()')
    coord_clean = [re.sub('\xa0', ' ', c) for c in coord]
    Coordonnees.append(coord_clean)
Coordonnees



pas de coordonnées
pas de coordonnées


In [144]:
for i , liste in enumerate(Coordonnees) : 
    if liste == [] : 
        Coordonnees[i] = ["NaN"]
Coordonnees

[['37° 58′ 00″ nord, 23° 43′ 00″ est'],
 ['48° 51′ 24″ nord, 2° 21′ 07″ est'],
 ['38° 38′ 53″ nord, 90° 12′ 44″ ouest'],
 ['37° 58′ 00″ nord, 23° 43′ 00″ est'],
 ['51° 30′ 26″ nord, 0° 07′ 39″ ouest'],
 ['59° 19′ 46″ nord, 18° 04′ 07″ est'],
 ['NaN'],
 ['51° 13′ nord, 4° 24′ est'],
 ['48° 51′ 24″ nord, 2° 21′ 07″ est'],
 ['52° 22′ 22″ nord, 4° 53′ 37″ est'],
 ['34° 03′ nord, 118° 15′ ouest'],
 ['NaN'],
 ['35° 41′ 22″ nord, 139° 41′ 30″ est'],
 ['51° 30′ 26″ nord, 0° 07′ 39″ ouest'],
 ['51° 30′ 26″ nord, 0° 07′ 39″ ouest'],
 ['60° 10′ 32″ nord, 24° 56′ 03″ est'],
 ['37° 48′ 51″ sud, 144° 58′ 06″ est'],
 ['41° 53′ 19″ nord, 12° 29′ 12″ est'],
 ['35° 41′ 22″ nord, 139° 41′ 30″ est'],
 ['19° 25′ 10″ nord, 99° 08′ 44″ ouest'],
 ['48° 09′ 00″ nord, 11° 34′ 30″ est'],
 ['45° 30′ 32″ nord, 73° 33′ 42″ ouest'],
 ['55° 45′ 09″ nord, 37° 37′ 23,11″ est'],
 ['34° 03′ nord, 118° 15′ ouest'],
 ['37° 33′ 57″ nord, 126° 58′ 41″ est'],
 ['41° 22′ 57″ nord, 2° 10′ 37″ est'],
 ['33° 45′ 25″ nord, 84° 23′

In [145]:
# Aplatir la liste et marquer les listes vides comme "vide"
Coordonnees_flat = [item if len(sublist) != 0 else "vide" for sublist in Coordonnees for item in sublist] + ["vide", "vide"]



df["Coordonnees"] = Coordonnees_flat
df

,Année,Olympiade,Ville hôte,Pays,Continent,Coordonnees
0,1896,I,Athènes,Grèce,Europe,"37° 58′ 00″ nord, 23° 43′ 00″ est"
1,1900,II,Paris,France,Europe,"48° 51′ 24″ nord, 2° 21′ 07″ est"
2,1904,III,Saint-Louis,États-Unis,Amérique du Nord,"38° 38′ 53″ nord, 90° 12′ 44″ ouest"
3,1906,•,Athènes,Grèce,Europe,"37° 58′ 00″ nord, 23° 43′ 00″ est"
4,1908,IV,Londres,Royaume-Uni,Europe,"51° 30′ 26″ nord, 0° 07′ 39″ ouest"
5,1912,V,Stockholm,Suède,Europe,"59° 19′ 46″ nord, 18° 04′ 07″ est"
6,1916,VI,Berlin,Allemagne,Europe,NaN
7,1920,VII,Anvers,Belgique,Europe,"51° 13′ nord, 4° 24′ est"
8,1924,VIII,Paris,France,Europe,"48° 51′ 24″ nord, 2° 21′ 07″ est"
9,1928,IX,Amsterdam,Pays-Bas,Europe,"52° 22′ 22″ nord, 4° 53′ 37″ est"


We then transform degree coordinates into numerical coordinates to build a map


In [149]:
import re

def dms2dd(degrees, minutes, seconds, direction):
    dd = float(degrees) + float(minutes)/60 + float(seconds)/(60*60);
    if direction in ('S', 'O'):
        dd *= -1
    return dd

def parse_dms(dms):
    parts = re.split('[^\d\w]+', dms)
    #potentially needs an adaptation
    #[student]
    lat = dms2dd(parts[0], parts[1], parts[2], parts[3])
    #lng = dms2dd(parts[4], parts[5], parts[6], parts[7])
    return lat

data['latitude'] = data['latitude'].apply(parse_dms)
#data['longitude'] = data['longitude'].apply(parse_dms)
print(data)

NameError: name 'data' is not defined

The map can be obtain with the following code

In [ ]:
import geopandas as gpd
from pathlib import Path
import folium
from IPython.display import display

gdf = gpd.GeoDataFrame(
    data, geometry=gpd.points_from_xy(data.longitude, data.latitude))

Path("leaflet").mkdir(parents=True, exist_ok=True)

center = gdf[['latitude', 'longitude']].mean().values.tolist()
sw = gdf[['latitude', 'longitude']].min().values.tolist()
ne = gdf[['latitude', 'longitude']].max().values.tolist()

m = folium.Map(location = center, tiles='openstreetmap')

# I can add marker one by one on the map
for i in range(0,len(gdf)):
    folium.Marker([gdf.iloc[i]['latitude'], gdf.iloc[i]['longitude']], popup=gdf.iloc[i]['name']).add_to(m)

m.fit_bounds([sw, ne])
display(m)

# Project

The objective of this project is to practice all concepts taught in the main lecture. Therefore, you will have to collect data around a thematic, identify a problematic, clean and format the data, provide some exploratory analysis and visualization, build models and evaluate them. You will have also to design dashboard and to storytell the whole pipeline.


The whole project will have two outputs describing the methodology and the results:
* A technical report targeting your datascientist colleagues
* An oral presentation targeting your CEO, chief, client. In this case, we assume that the audience is not specialized in data science. (But you also need to present the methodology to convince them to trust the results).


**Requests:**
* Team work of two people (same group throughout the semester)
* All your work should be stored on a git repo: [tutorial](https://github.com/baskiotisn/2IN013robot2023/blob/d979333fb80c9b6acd9515aaec040943d10d365c/docs/tutoriel_git.pdf)

**Remarks:**
There are also other libraries or issues you'll encounter when collecting data. Some of these are listed below, but don't be shy and interact with a search engine to solve your own issue!
* [Regular expressions](https://docs.python.org/3/howto/regex.html) might be useful!
* [API with authentication](https://www.geeksforgeeks.org/authentication-using-python-requests/)
*   [Selenium](https://selenium-python.readthedocs.io/) when pages are generated wia javascript scripts
* [Playwright](https://playwright.dev/) --> looks easier and more adapted than Selenium
* [Scrapy](https://scrapy.org/) for web crawling / or when you don't know the URL.[Tuto here](https://doc.scrapy.org/en/latest/intro/tutorial.html)
* [Summary of some difficulties in scrapping data from the web](https://www.zenrows.com/blog/web-scraping-challenges#page-structure-changes)

## Your daily task
Collecting data from the web and open data portals. Open data might include csv files, you can use them, but be aware that scraping should be your main acitivity in the dataset gathering.
You can begin to format the data, merge them to build dataframe that would be analyzed next week.